In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor

In [2]:
X_train = pd.read_csv(r"C:\Users\user\Desktop\daniel\DATA SCIENCE\HENRY\HENRY BOOTCAMP\HACKATON\Hackathon\Housing Dreams\house_train_raw.csv")
X_test = pd.read_csv(r"C:\Users\user\Desktop\daniel\DATA SCIENCE\HENRY\HENRY BOOTCAMP\HACKATON\Hackathon\Housing Dreams\houses_test_raw.csv")
df_list = [X_train,X_test]

In [6]:
df = pd.concat(df_list)
df.set_index(df["Id"],inplace=True)
df.drop(columns=["Id"],inplace=True)
df

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500.0
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500.0
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500.0
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000.0
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2915,160,RM,21.0,1936,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,6,2006,WD,Normal,NaN
2916,160,RM,21.0,1894,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,4,2006,WD,Abnorml,NaN
2917,20,RL,160.0,20000,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2006,WD,Abnorml,NaN


In [7]:
df.isnull().sum()[df.isnull().sum() > 0]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
SalePrice       1459
dtype: int64

In [8]:
cols_with_high_nan = [col for col in df.columns if df[col].isnull().sum() > 500]
cols_with_high_nan

['Alley', 'FireplaceQu', 'PoolQC', 'Fence', 'MiscFeature', 'SalePrice']

In [9]:
#Ahora revisaremos el campo MiscFeature
#Revisando encontramos que este campo cuando tiene valores nulos significa que la casa no tiene caracteristicas distintas
df.loc[df["MiscFeature"].isnull(), "MiscFeature"] = "None"
#Ahora revisaremos el campo Alley
#Revisando encontramos que en este campo los valores nulos representan una casa sin acceso por callejon, asi que seteamos el valor "None"
df.loc[df["Alley"].isnull(), "Alley"] = "None"
#Ahora, al investigar la columna Fence, comprobamos que cuando hay valores nulos, significa que la casa no tiene Valla. Asi que arreglamos esto
df.loc[df["Fence"].isnull(), "Fence"] = "None"
#Ahora, al investigar la columna FireplaceQu, comprobamos que cuando hay valores nulos, significa que la casa no tiene Chimenea. Asi que arreglamos esto
df.loc[df["FireplaceQu"].isnull(), "FireplaceQu"] = "None"


In [19]:
#NA en PoolQC significa que la casa no tiene piscina, eso explica la gran cantidad de valores faltantes:
df.loc[df["PoolQC"].isnull(), "PoolQC"] = "None"

Investigando LotFrontage, no se tiene una razon especifica por la cual falten valores en este campo, asi que idearemos un metodo para imputar sus valores faltantes

In [10]:
#Verificamos los outliers de LotFrontage:
px.box(df,y="LotFrontage")

In [11]:
#LotFrontage tiene outliers por lo tanto me deshare de ellos para mi imputacion de valores nulos
q3, q1 = np.nanpercentile(np.array(X_train.LotFrontage.values), [75,25])
iqr = q3-q1
h_outliers, l_outliers = (q3 + 1.5*iqr), (q1 - 1.5*iqr)


In [12]:
fig = px.box(df[(df.LotFrontage < h_outliers) & (df.LotFrontage > l_outliers) ],x="Neighborhood",y="LotFrontage")
fig.show()

In [13]:
#Aqui agrupo por neighborhood para sacar la media de LotFrontage y asi crear un diccionario de 
median_lotfront_per_neighbour = pd.DataFrame(df[(df.LotFrontage < h_outliers) & (df.LotFrontage > l_outliers) ].groupby("Neighborhood")["LotFrontage"].median())
map_dict = {median_lotfront_per_neighbour.LotFrontage.index[i] : median_lotfront_per_neighbour.LotFrontage.values[i] for i in range(len(median_lotfront_per_neighbour.LotFrontage.index))}
df.loc[df.LotFrontage.isnull(),"LotFrontage"] = df[df.LotFrontage.isnull()].Neighborhood.map(map_dict)

## Imputacion de las demas variables

In [15]:
df.isna().sum()[df.isna().sum()>0]

MSZoning           4
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
SaleType           1
SalePrice       1459
dtype: int64

In [16]:
#Hay 160 valores nulos en las columnas relacionadas con el garage, revisando el diccionario, verificamos que se debe a que estas casas no tienen un garage, asi que imputamos los valores nulos por NONE
for col in ["GarageType","GarageFinish","GarageQual","GarageCond"]:
    df.loc[df[col].isnull(),col] = "No Garage"
df.loc[df["GarageYrBlt"].isnull(),"GarageYrBlt"] = 0
#Perfecto, hemos imputado los valores faltantes de estas columnas.

In [17]:
#Se realizara el mismo procedimiento para los campos "Basement"
for col in ["BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2"]:
    df.loc[df[col].isnull(),col] = "No Basement"
df.isna().sum()[df.isna().sum()>0]

MSZoning           4
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
GarageCars         1
GarageArea         1
PoolQC          2909
SaleType           1
SalePrice       1459
dtype: int64

In [20]:
#Para el campo MasVnrType verificamos que los campos con NA es porque no tienen chapa de mamposteria
df.loc[df["MasVnrType"].isnull(),"MasVnrType"] = "None"
df.loc[df["MasVnrArea"].isnull(),"MasVnrArea"] = 0
df.isna().sum()[df.isna().sum()>0]

MSZoning           4
Utilities          2
Exterior1st        1
Exterior2nd        1
BsmtFinSF1         1
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
GarageCars         1
GarageArea         1
SaleType           1
SalePrice       1459
dtype: int64

In [25]:
cols_na_left = df.isna().sum()[df.isna().sum()>0].index.tolist()
cols_na_left.remove("SalePrice")
df.dropna(subset=cols_na_left,inplace=True)
df.isna().sum()[df.isna().sum()>0]

SalePrice    1447
dtype: int64

## Codificacion de Variables categoricas 

In [27]:
#Hice una exploracion de cada campo, verificando si es una variable ordinal o nominal, ya que hay diferencia entre este tipo de variables categoricas
#cada una se codifica de una manera distinta. Las variables que pueden ser ordenadas las almacene en "ordinal_cols"
from sklearn.preprocessing import LabelEncoder
ordinal_cols = ["Street", "LotShape", "LandContour", "Utilities", "LandSlope", "MSSubClass", "OverallQual","OverallCond","ExterQual","ExterCond","BsmtQual","BsmtCond","BsmtExposure","BsmtFinType1","BsmtFinType2","HeatingQC","CentralAir","Electrical","KitchenQual","FireplaceQu","GarageFinish","GarageQual","GarageCond","PoolQC","Fence"]
#Ahora se transforman:
for col in ordinal_cols:
    le = LabelEncoder()
    le.fit(list(df[col].values)) 
    df[col] = le.transform(list(df[col].values))

In [28]:
#Variables categoricas nominales:
categorical_variables = [col for col in df.columns if df[col].dtype == "object"]
len(categorical_variables)

21

In [29]:
#Removemos nuestra variable target del dataframe
y_train = df.pop("SalePrice")

In [30]:
#Importamos lo necesario para realizar el One Hot encoder:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer([('ohe', OneHotEncoder(handle_unknown="ignore",sparse=False), categorical_variables)],#Los nuevos valores que se encuentren en las columnas de test, los ignorará
                         remainder='passthrough') #transformamos las columnas categoricas, y las demas cols las pasamos de largo
                         
df_encoded = pd.DataFrame(ct.fit_transform(df), columns=ct.get_feature_names_out())
df_encoded.index = df.index 

In [31]:
print(df.shape,df_encoded.shape)

(2906, 79) (2906, 224)


In [42]:
#Inserto mi variable "y" solamente para separar el dataframe en test y en train 
df_encoded.insert(0,"y",y_train)

In [45]:
#Separlo los dataframes con ayuda de la variable "y" que inserté
#La parte del df donde "y" es nula es el conjunto de TEST y donde no es nula, es el conjunto de TRAINING
X_train = df_encoded[~df_encoded.y.isnull()]
X_test = df_encoded[df_encoded.y.isnull()]
#Establezo mi variable target con su longitud correcta:
y_train = X_train["y"]
#Dropeo de ambos df's la variable para no cometer ningun error
X_train.drop(columns=["y"],inplace=True)
X_test.drop(columns=["y"],inplace=True)

(1459, 224) (1447, 224)


C:\Users\user\AppData\Local\Temp\ipykernel_2432\108778292.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\user\AppData\Local\Temp\ipykernel_2432\108778292.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## Creacion del modelo y obtencion del error

In [64]:
#Instanciamos el modelo, y seteamos los parametros que consideramos apropiados:
xgb = XGBRegressor(n_estimators=1200,learning_rate=0.06)
xgb.fit(X_train,y_train)
#Predecimos sobre la data de train para ver la metrica:
predictions = xgb.predict(X_train)
print(mean_squared_log_error(y_train,predictions,squared=False))

0.0016918221728300436


In [65]:
#Ahora predecimos sobre el conjunto de testing (El que realmente nos importa)
test_predictions = xgb.predict(X_test)
len(test_predictions)

1447

In [68]:
#Genero un dataset de mis predicciones y agrego el indice, para identificar cada registro, ya que se dropearon algunos valores (muy pocos)
test_predictions = pd.DataFrame(test_predictions,columns=["preds"])
test_predictions.insert(0,"Index",X_test.index.values)

In [70]:
#Guardo mis predicciones en un csv
test_predictions.to_csv(r"C:\Users\user\Desktop\daniel\DATA SCIENCE\HENRY\HENRY BOOTCAMP\HACKATON\Hackathon\Housing Dreams\preds.csv")